In [33]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, auc, confusion_matrix, average_precision_score, precision_recall_curve


## Importing Data

In [92]:
goal_11 = pd.read_excel("DataVizChallenge2021/sdg_data/Goal11.xlsx")

In [93]:
goal_11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19296 entries, 0 to 19295
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Goal               19296 non-null  int64  
 1   Target             19296 non-null  object 
 2   Indicator          19296 non-null  object 
 3   SeriesCode         19296 non-null  object 
 4   SeriesDescription  19296 non-null  object 
 5   GeoAreaCode        19296 non-null  int64  
 6   GeoAreaName        19296 non-null  object 
 7   TimePeriod         19296 non-null  int64  
 8   Value              19296 non-null  float64
 9   Time_Detail        19296 non-null  object 
 10  TimeCoverage       0 non-null      float64
 11  UpperBound         0 non-null      float64
 12  LowerBound         0 non-null      float64
 13  BasePeriod         0 non-null      float64
 14  Source             19296 non-null  object 
 15  GeoInfoUrl         0 non-null      float64
 16  FootNote           152

In [94]:
subset_goal_11 = goal_11[['GeoAreaName', 'SeriesCode', 'Value']]

In [95]:
goal_11_transformed = subset_goal_11.pivot_table(index='GeoAreaName', columns = 'SeriesCode', aggfunc='mean')

In [96]:
goal_11_transformed.loc[['World']]

Value                                                       \
SeriesCode  EN_ATM_PM25 EN_LND_SLUM EN_REF_WASCOL SD_CPA_UPRDP SG_DSR_LGRGSR   
GeoAreaName                                                                    
World         34.708362        23.5           NaN        156.0           NaN   

                                                                           \
SeriesCode  SG_DSR_SFDRR SG_DSR_SILN SG_DSR_SILS SG_GOV_LOGV VC_DSR_AFFCT   
GeoAreaName                                                                 
World              118.0         NaN         NaN         NaN          NaN   

             ...                                                               \
SeriesCode   ... VC_DSR_IJILN VC_DSR_LSGP VC_DSR_MISS VC_DSR_MMHN VC_DSR_MORT   
GeoAreaName  ...                                                                
World        ...          NaN         NaN         NaN         NaN         NaN   

                                                                         
SeriesCode  VC_DSR_MTMP VC_DSR_OBDN VC_DSR_PDAN VC_DSR_PDLN VC_DSR_PDYN  
GeoAreaName                                                              
World               NaN         NaN         NaN         NaN         NaN  

[1 rows x 34 columns]

In [97]:
goal_11_T_noWorld = goal_11_transformed.drop(index='World')

In [98]:
drop_features = [('Value', 'VC_DSR_OBDN'),('Value', 'VC_DSR_CDYN'),('Value', 'SG_DSR_SFDRR'),('Value', 'VC_DSR_DDPA'),('Value', 'SG_DSR_SFDRR'),('Value', 'VC_DSR_CHLN'), ('Value', 'VC_DSR_CHLN')]

In [99]:
draft = goal_11_T_noWorld.drop(columns=drop_features)

In [100]:
draft.info()

<class 'pandas.core.frame.DataFrame'>
Index: 265 entries, Afghanistan to Åland Islands
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   (Value, EN_ATM_PM25)    202 non-null    float64
 1   (Value, EN_LND_SLUM)    133 non-null    float64
 2   (Value, EN_REF_WASCOL)  89 non-null     float64
 3   (Value, SD_CPA_UPRDP)   264 non-null    float64
 4   (Value, SG_DSR_LGRGSR)  132 non-null    float64
 5   (Value, SG_DSR_SILN)    107 non-null    float64
 6   (Value, SG_DSR_SILS)    106 non-null    float64
 7   (Value, SG_GOV_LOGV)    110 non-null    float64
 8   (Value, VC_DSR_AFFCT)   142 non-null    float64
 9   (Value, VC_DSR_AGLH)    102 non-null    float64
 10  (Value, VC_DSR_BSDN)    99 non-null     float64
 11  (Value, VC_DSR_CDAN)    80 non-null     float64
 12  (Value, VC_DSR_CILN)    90 non-null     float64
 13  (Value, VC_DSR_DAFF)    142 non-null    float64
 14  (Value, VC_DSR_EFDN)    77 

In [101]:
#dropping countries that don't have any data for %in urban slums
goal_11_final = draft[draft[('Value', 'EN_LND_SLUM')].notna()]

In [102]:
goal_11_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 133 entries, Afghanistan to Zimbabwe
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   (Value, EN_ATM_PM25)    129 non-null    float64
 1   (Value, EN_LND_SLUM)    133 non-null    float64
 2   (Value, EN_REF_WASCOL)  61 non-null     float64
 3   (Value, SD_CPA_UPRDP)   133 non-null    float64
 4   (Value, SG_DSR_LGRGSR)  81 non-null     float64
 5   (Value, SG_DSR_SILN)    64 non-null     float64
 6   (Value, SG_DSR_SILS)    64 non-null     float64
 7   (Value, SG_GOV_LOGV)    67 non-null     float64
 8   (Value, VC_DSR_AFFCT)   96 non-null     float64
 9   (Value, VC_DSR_AGLH)    73 non-null     float64
 10  (Value, VC_DSR_BSDN)    68 non-null     float64
 11  (Value, VC_DSR_CDAN)    60 non-null     float64
 12  (Value, VC_DSR_CILN)    66 non-null     float64
 13  (Value, VC_DSR_DAFF)    96 non-null     float64
 14  (Value, VC_DSR_EFDN)    58 non-n

In [103]:
# Setting Percent of Urban Population Living in Slums as Target to predict
target = np.array(goal_11_final[('Value', 'EN_LND_SLUM')])

#Remove Target from features
features = goal_11_final.drop(('Value', 'EN_LND_SLUM'), axis=1)

#Saving Feature names for future use
feature_list = list(features.columns)

#Converting to numpy array
features = np.array(features)


In [104]:
# Attempting to Impute Missing Values
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [105]:
#Using Iterative imputer to fill in null-values

imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(features)

IterativeImputer(add_indicator=False, estimator=None,
                 imputation_order='ascending', initial_strategy='mean',
                 max_iter=10, max_value=None, min_value=None,
                 missing_values=nan, n_nearest_features=None, random_state=0,
                 sample_posterior=False, skip_complete=False, tol=0.001,
                 verbose=0)

In [106]:
features_imputed = imp.transform(features)

In [107]:
# Using Skicit-learn to split data into training and testing sets
# Split the data into training and testing sets
train_features, test_features, train_target, test_target = train_test_split(
    features_imputed, target, test_size = 0.25, random_state = 42)

In [108]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_target.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_target.shape)

Training Features Shape: (99, 28)
Training Labels Shape: (99,)
Testing Features Shape: (34, 28)
Testing Labels Shape: (34,)


In [42]:
# # The baseline predictions are the historical averages
# baseline_preds = test_features[:, feature_list.index('average')]
# # Baseline errors, and display average baseline error
# baseline_errors = abs(baseline_preds - test_labels)
# print('Average baseline error: ', round(np.mean(baseline_errors), 2))

In [109]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_target);

In [116]:
target_predict = rf.predict(test_features)

In [112]:
print("Training Accuracy = ", rf.score(train_features, train_target))
print("Test Accuracy = ", rf.score(test_features, test_target))

Training Accuracy =  0.8900611611657318
Test Accuracy =  0.3788009275153998


In [113]:
from sklearn.metrics import mean_squared_error

In [117]:
mean_squared_error(test_target,target_predict)

395.3664785056347

Horrible test accuracy compared to training set, possibly too many estimators. Adjusting parameters of the model 

In [124]:
# Instantiate model with 1000 decision trees
rf2 = RandomForestRegressor(n_estimators = 1000, random_state = 42, oob_score=True, max_leaf_nodes = 5, max_depth = 10)
# Train the model on training data
rf2.fit(train_features, train_target);
target_predict = rf2.predict(test_features);
mean_squared_error(test_target,target_predict)


411.4166493568787

In [127]:
f_importance  = list(rf.feature_importances_)

In [129]:
list(zip(feature_list, f_importance))

[(('Value', 'EN_ATM_PM25'), 0.19703193725021798),
 (('Value', 'EN_REF_WASCOL'), 0.11068336555983779),
 (('Value', 'SD_CPA_UPRDP'), 0.009215792280332162),
 (('Value', 'SG_DSR_LGRGSR'), 0.032573541620382514),
 (('Value', 'SG_DSR_SILN'), 0.021599461177760397),
 (('Value', 'SG_DSR_SILS'), 0.03131437108505413),
 (('Value', 'SG_GOV_LOGV'), 0.056301385310143705),
 (('Value', 'VC_DSR_AFFCT'), 0.01769174725911507),
 (('Value', 'VC_DSR_AGLH'), 0.010413641548559302),
 (('Value', 'VC_DSR_BSDN'), 0.027001282312201967),
 (('Value', 'VC_DSR_CDAN'), 0.015122744600635686),
 (('Value', 'VC_DSR_CILN'), 0.019072732380534873),
 (('Value', 'VC_DSR_DAFF'), 0.019040853671713487),
 (('Value', 'VC_DSR_EFDN'), 0.01721833556275882),
 (('Value', 'VC_DSR_ESDN'), 0.016497745958525278),
 (('Value', 'VC_DSR_GDPLS'), 0.009889780677424553),
 (('Value', 'VC_DSR_HFDN'), 0.02249213262852094),
 (('Value', 'VC_DSR_HOLH'), 0.012749909328213335),
 (('Value', 'VC_DSR_HSDN'), 0.06644236644288837),
 (('Value', 'VC_DSR_IJILN'), 0.

In [132]:
sorted(list(zip(feature_list, f_importance)), key=lambda x: x[1], reverse=True)

[(('Value', 'EN_ATM_PM25'), 0.19703193725021798),
 (('Value', 'EN_REF_WASCOL'), 0.11068336555983779),
 (('Value', 'VC_DSR_HSDN'), 0.06644236644288837),
 (('Value', 'SG_GOV_LOGV'), 0.056301385310143705),
 (('Value', 'VC_DSR_LSGP'), 0.05292261560935805),
 (('Value', 'VC_DSR_MMHN'), 0.04383317309917291),
 (('Value', 'VC_DSR_PDLN'), 0.04007153358708921),
 (('Value', 'VC_DSR_MTMP'), 0.03921400482932794),
 (('Value', 'SG_DSR_LGRGSR'), 0.032573541620382514),
 (('Value', 'SG_DSR_SILS'), 0.03131437108505413),
 (('Value', 'VC_DSR_BSDN'), 0.027001282312201967),
 (('Value', 'VC_DSR_MISS'), 0.026410062167164208),
 (('Value', 'VC_DSR_IJILN'), 0.025297223077617222),
 (('Value', 'VC_DSR_PDYN'), 0.024356048529515745),
 (('Value', 'VC_DSR_HFDN'), 0.02249213262852094),
 (('Value', 'SG_DSR_SILN'), 0.021599461177760397),
 (('Value', 'VC_DSR_MORT'), 0.021050011967916338),
 (('Value', 'VC_DSR_CILN'), 0.019072732380534873),
 (('Value', 'VC_DSR_DAFF'), 0.019040853671713487),
 (('Value', 'VC_DSR_AFFCT'), 0.0176